<a href="https://colab.research.google.com/github/zahra-sharifzadeh/pytorch/blob/main/06_PyTorch_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>